In [1]:
! pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 1.4 MB/s eta 0:00:00


### Import libraries

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root_path = "/content/drive/MyDrive/Deep_learning_course/Video_processing/"
data_path = "/content/data/"

In [8]:
import os
import torch
from torcheval.metrics.functional import multiclass_accuracy, multiclass_auroc, multiclass_recall
import torch.nn as nn
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import pickle
from tqdm.notebook import tqdm

In [10]:
import sys
sys.path.append(root_path + "src")
from dataloader import load_test_data
from video_sequence_model import VideoSequenceModel
from fit_func import FitTrainEval

In [ ]:
external_data_path = os.path.join(root_path, "data/", "UFC-5.zip")
local_data_path = data_path
!unzip -q $external_data_path -d $local_data_path

### Load Data

In [12]:
# data parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_folder_path = os.path.join(data_path, "UFC-5/")
batch_size = 5
num_workers = 8
resize_size = 256
nframes = 10
# model parameters
embedding_size = 300
lstm_hidden = 300
lstm_layers = 5
num_classes = 5
model_load = "seqvideo_4.pkl"
model_load = os.path.join(root_path, "models/", model_load)
metrics = [multiclass_accuracy]
EPOCHS = 500

In [13]:
data_test = load_test_data(
    path_to_data=data_folder_path,
    batch_size=batch_size,
    resize=resize_size,
    nframes=nframes,
    workers=num_workers,
    pin_memory_device=device
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


### Train models

In [20]:
model = VideoSequenceModel(
    sequence_length=nframes,
    embedding_size=embedding_size,
    lstm_hidden=lstm_hidden,
    lstm_layers=lstm_layers,
    num_classes=num_classes
)
state = torch.load(model_load, map_location=torch.device('cpu'))
model.load_state_dict(state["model"])
model = model.to(device)

### Test model

In [28]:
def test_model(model, data_test):
  model.eval()
  with torch.no_grad():
    for i, batch in tqdm(enumerate(data_test), total=len(data_test)):
      batch_x = batch["sequence"].to(device)
      batch_y = batch["target"].to(device)

      y_pred = model(batch_x)
      accuracy = multiclass_accuracy(y_pred, batch_y).item()
      recall = multiclass_accuracy(y_pred, batch_y, num_classes=5, average=None)
      auroc = multiclass_auroc(y_pred, batch_y, num_classes=5).item()

    print(f"accuracy: {accuracy}")
    print(f"auroc: {auroc}")
    print(f"recall: {recall}")

  return accuracy, recall, auroc

In [29]:
acc, rec, aurc = test_model(model, data_test)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/7 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


accuracy: 1.0
auroc: 0.800000011920929
recall: tensor([1., nan, 1., 1., nan])


In [ ]:
0.8, 0.8
1, 0.7
1, 0.9
1, 0.8